In [ ]:
!pip install --upgrade requests 
!pip install --upgrade beautifulsoup4
!pip install --upgrade lxml 
!pip install --upgrade pip

In [1]:
import requests #module to request html 
import bs4 #module to parse html code
import time #introduce pauses in script (tricks website into thinking we are not a robot)

In [2]:
#components of an advanced search on AllAfrica
root_url = "http://allafrica.com/search/advanced.html"
search = {'q': '', #main search term
            'g': 'all', 
            'p': '', 
            'c': '', #category, can specify country name or other variables
            'st': '2',  #1 sorts articles by relevance, 2 sorts by chronological order, 3 sorts by reverse chronological order
            'di': '1', 
            'sd': '19961017', #start date
            'ed': '20150702', #end date
            'os': '0'}     #page number

In [3]:
url = "http://allafrica.com/search/advanced.html?q=arrest&g=all&c=corruption&p=&sd=19961017&ed=20150702&st=2&di=1&os={}"
#the curly brackets at the end {} are CRUCIAL. When you pass this string object to .format(), whatever is in the parenthesis is inserted in the curly brackets.

article_urls = []
for i in range(0, 2901, 25):  #perform an advanced search on AllAfrica, see how many results you have (e.g. 2901), and note that 25 results are presented per page
    time.sleep(1) #pause 1 sec every time you visit new page
    response = requests.get(url.format(i)) # sends out request using different search terms criteria 
    soup = bs4.BeautifulSoup(response.text) #stores search as a BeautifulSoup object
    links = [a.attrs.get('href') for a in soup.select('ul.stories a[href^=/stories]')] #selects story links within BeautifulSoup object
    article_urls.extend(links)
print article_urls

/Users/Fiona_Shen_Bayh/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


ConnectionError: ('Connection aborted.', BadStatusLine("''",))

In [ ]:
def get_article_data(article_url):
    response = requests.get('http://allafrica.com' + article_url)
    soup = bs4.BeautifulSoup(response.text)
    article_data = {}
    article_data['title'] = soup.select('body > div.columns > div > div.column.main > div > div.section.article > div > div.heading > h1')[0].get_text().encode(encoding='utf-8')  
    article_data['date'] = soup.select('body > div.columns > div > div.column.main > div > div.section.article > div > div.heading > div.publication-date')[0].get_text()
    return article_data

In [ ]:
articles = []

for article_url in article_urls:
    articles.append(get_article_data(article_url)) #add individual article data to the articles list
    
#Note, this cell can take several hours to run

In [ ]:
import csv

def WriteDictToCSV(csv_file,csv_columns,dict_data):
    with open(csv_file, 'w') as csvfile: 
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns) 
        writer.writeheader()
        writer.writerows(dict_data)
    return

csv_columns = ["title", "date"] #specify the column headings --> make sure they are the same names as the dictionary keys in articles
csv_file = "Desktop/allafrica_arrest.csv" #this saves file to desktop
dict_data = articles

WriteDictToCSV(csv_file, csv_columns, dict_data)